# FLAN-T5 Finetuning for QA on Yahoo Data

Code inspired from Taughdata tutorial

https://www.toughdata.net/blog/post/finetune-flan-t5-question-answer-quora-dataset

## Prerequisites

## Import the libraries

In [1]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

/ibex/user/shaima0d/KSL_Trainings/hpc-saudi-2024/ds-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Loading

In [2]:
from datasets import load_dataset

In [3]:
# Acquire the training data from Hugging Face
yahoo_answers_qa = load_dataset("yahoo_answers_qa")

In [4]:
# Check the length of the data and its structure
yahoo_answers_qa

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
        num_rows: 87362
    })
})

In [5]:
"""# Take a subset of the original dataset
SAMPLE_SIZE = 3000

subset_yahoo_answers_qa = yahoo_answers_qa.select(range(SAMPLE_SIZE))

subset_yahoo_answers_qa"""

'# Take a subset of the original dataset\nSAMPLE_SIZE = 3000\n\nsubset_yahoo_answers_qa = yahoo_answers_qa.select(range(SAMPLE_SIZE))\n\nsubset_yahoo_answers_qa'

In [6]:
SAMPLE_SIZE = 3000
subset_yahoo_answers_qa = yahoo_answers_qa['train'].select(range(SAMPLE_SIZE))

In [7]:
subset_yahoo_answers_qa = subset_yahoo_answers_qa.train_test_split(test_size=0.3)

In [8]:
subset_yahoo_answers_qa

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
        num_rows: 2100
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
        num_rows: 900
    })
})

In [9]:
# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME,legacy=False,clean_up_tokenization_spaces=False)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [10]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question: "

# Define the preprocessing function

def preprocess_function(examples):
    """Add prefix to the sentences, tokenize the text, and set the labels"""
    # The "inputs" are the tokenized answer:
    inputs = [prefix + doc for doc in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    # The "labels" are the tokenized outputs:
    labels = tokenizer(text_target=examples["answer"], max_length=512, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
# Map the preprocessing function across our dataset
tokenized_dataset = subset_yahoo_answers_qa.map(preprocess_function, batched=True)

Map: 100%|██████████| 900/900 [00:00<00:00, 2792.89 examples/s]


In [12]:
# Set up Rouge score for evaluation
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

In [13]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return result

In [14]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 1

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=L_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
    weight_decay=WEIGHT_DECAY,
    save_total_limit=SAVE_TOTAL_LIM,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    push_to_hub=False
)

In [15]:
# Set up trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [16]:
print(f'Number of training itreations: {NUM_EPOCHS*len(subset_yahoo_answers_qa["train"])/BATCH_SIZE}')

Number of training itreations: 262.5


In [17]:
# Trigger the model training
trainer.train()

Using port 42789
[2024-12-06 15:40:16,995] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.


/ibex/user/shaima0d/KSL_Trainings/hpc-saudi-2024/ds-env/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/ibex/user/shaima0d/KSL_Trainings/hpc-saudi-2024/ds-env/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/ibex/user/shaima0d/KSL_Trainings/hpc-saudi-2024/ds-env/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,3.118209,0.180254,0.036570,0.142656,0.161268


/ibex/user/shaima0d/KSL_Trainings/hpc-saudi-2024/ds-env/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=263, training_loss=3.414285521996792, metrics={'train_runtime': 131.8758, 'train_samples_per_second': 15.924, 'train_steps_per_second': 1.994, 'total_flos': 78233527517184.0, 'train_loss': 3.414285521996792, 'epoch': 1.0})

In [18]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!zip -r results.zip results

  adding: results/ (stored 0%)
  adding: results/checkpoint-3500/ (stored 0%)
  adding: results/checkpoint-3500/model.safetensors (deflated 7%)
  adding: results/checkpoint-3500/special_tokens_map.json (deflated 85%)
  adding: results/checkpoint-3500/tokenizer_config.json (deflated 95%)
  adding: results/checkpoint-3500/scheduler.pt (deflated 55%)
  adding: results/checkpoint-3500/spiece.model (deflated 48%)
  adding: results/checkpoint-3500/rng_state.pth (deflated 25%)
  adding: results/checkpoint-3500/training_args.bin (deflated 51%)
  adding: results/checkpoint-3500/added_tokens.json (deflated 83%)
  adding: results/checkpoint-3500/trainer_state.json (deflated 65%)
  adding: results/checkpoint-3500/generation_config.json (deflated 30%)
  adding: results/checkpoint-3500/optimizer.pt (deflated 12%)
  adding: results/checkpoint-3500/config.json (deflated 62%)
  adding: results/runs/ (stored 0%)
  adding: results/runs/Dec06_15-26-20_gpu213-02/ (stored 0%)
  adding: results/runs/Dec06_15

In [ ]:
!ls results

## Run in inference mode

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

#last_checkpoint = "./results/checkpoint-22500"
last_checkpoint = "./results/checkpoint-3500"
finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

In [ ]:
my_question = "What do you think about the benefit of Artificial Intelligence?"
inputs = "Please answer to this question: " + my_question

inputs = tokenizer(inputs, return_tensors="pt")

In [ ]:
outputs = finetuned_model.generate(**inputs)
answer = tokenizer.decode(outputs[0])

print("Answer:", fill(answer, width=80))